In [0]:
from pyspark.sql.functions import when

In [0]:
dbutils.widgets.text("year","")
dbutils.widgets.text("month","")
dbutils.widgets.text("day","")
dbutils.widgets.text("storageName","")

In [0]:
year = dbutils.widgets.get("year")
month = dbutils.widgets.get("month")
day = dbutils.widgets.get("day")
storageName = dbutils.widgets.get("storageName")

In [0]:
display(dbutils.fs.ls(f"abfss://raw@{storageName}.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://raw@mateuszlegoprod.dfs.core.windows.net/downloadList/,downloadList/,0,1719593647000
abfss://raw@mateuszlegoprod.dfs.core.windows.net/files/,files/,0,1719593776000
abfss://raw@mateuszlegoprod.dfs.core.windows.net/user_files/,user_files/,0,1719594987000
abfss://raw@mateuszlegoprod.dfs.core.windows.net/usersList/,usersList/,0,1719593720000


In [0]:
df = spark.read.option("inferSchema","true").option("header","true").csv(f"abfss://raw@{storageName}.dfs.core.windows.net/files/minifigs/Year={year}/Month={month}/Day={day}/*.csv.gz")

In [0]:
df.show()

+----------+--------------------+---------+--------------------+
|   fig_num|                name|num_parts|             img_url|
+----------+--------------------+---------+--------------------+
|fig-000001|  Toy Store Employee|        4|https://cdn.rebri...|
|fig-000002|        Customer Kid|        4|https://cdn.rebri...|
|fig-000003|Assassin Droid, W...|        8|https://cdn.rebri...|
|fig-000004|Man, White Torso,...|        4|https://cdn.rebri...|
|fig-000005|Captain America w...|        3|https://cdn.rebri...|
|fig-000006|        Lloyd Avatar|        5|https://cdn.rebri...|
|fig-000007|Lloyd / Digi Lloy...|        5|https://cdn.rebri...|
|fig-000008|        Wonder Woman|        4|https://cdn.rebri...|
|fig-000009|           The Flash|        4|https://cdn.rebri...|
|fig-000010|Batman, Dark Blui...|        5|https://cdn.rebri...|
|fig-000011|        Captain Cold|        4|https://cdn.rebri...|
|fig-000012|        Truck Driver|        4|https://cdn.rebri...|
|fig-000013|       Gorill

In [0]:
df = df.withColumnRenamed("fig_num","figID").withColumnRenamed("name", "figName").withColumnRenamed("num_parts","partsCount").withColumnRenamed("img_url","imgURL")

In [0]:
df = df.withColumn("Complexity", when(df.partsCount <=4, "Low").when(df.partsCount <=7, "Medium").otherwise("High"))

In [0]:
df.show()

+----------+--------------------+----------+--------------------+----------+
|     figID|             figName|partsCount|              imgURL|Complexity|
+----------+--------------------+----------+--------------------+----------+
|fig-000001|  Toy Store Employee|         4|https://cdn.rebri...|       Low|
|fig-000002|        Customer Kid|         4|https://cdn.rebri...|       Low|
|fig-000003|Assassin Droid, W...|         8|https://cdn.rebri...|      High|
|fig-000004|Man, White Torso,...|         4|https://cdn.rebri...|       Low|
|fig-000005|Captain America w...|         3|https://cdn.rebri...|       Low|
|fig-000006|        Lloyd Avatar|         5|https://cdn.rebri...|    Medium|
|fig-000007|Lloyd / Digi Lloy...|         5|https://cdn.rebri...|    Medium|
|fig-000008|        Wonder Woman|         4|https://cdn.rebri...|       Low|
|fig-000009|           The Flash|         4|https://cdn.rebri...|       Low|
|fig-000010|Batman, Dark Blui...|         5|https://cdn.rebri...|    Medium|

In [0]:
df.write.format("delta").mode("overwrite").save(f"abfss://curated@{storageName}.dfs.core.windows.net/files/minifigs/")